## Tutorial Example #01

### 1. Using a Ready-Made CNN model (Keras):

In [ ]:
# Install if necessary.
#!pip install keras

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings                   
from sklearn.model_selection import train_test_split
from keras.datasets.cifar10 import load_data                           
from keras.models import Model                       # We will use Functional API.
from keras.layers import Dense, Dropout, Flatten
from keras.applications.mobilenet import MobileNet   # We will use a relatively small neural network.
from keras.applications.xception import Xception
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50
from keras.utils import to_categorical
from keras.optimizers import Adam, RMSprop, SGD
warnings.filterwarnings('ignore')                    # Turn the warnings off.
%matplotlib inline

#### 1.1. Read in the data:

More information about the dataset can be found [here](https://www.cs.toronto.edu/~kriz/cifar.html).

In [ ]:
# Bring in the data.
(X_train, y_train), (X_test, y_test) = load_data()
n_train_size = X_train.shape[0]

In [ ]:
# View the shapes.
print("-"*50)
print("Training data X shape: {}".format(X_train.shape))
print("Training data y shape: {}".format(y_train.shape))
print("-"*50)
print("Test data X shape: {}".format(X_test.shape))
print("Test data y shape: {}".format(y_test.shape))
print("-"*50)

#### 1.2. Visualize the data:

In [ ]:
i_image = 123                                   # You may change this at will. 
plt.imshow(X_train[i_image,:,:])
plt.show()

#### 1.3. Prepare the data:

In [ ]:
# Scaling.
X_train = X_train/255                   
X_test = X_test/255                        

In [ ]:
# Reshaping.
X_train = X_train.reshape(-1,32,32,3)
X_test = X_test.reshape(-1,32,32,3)

In [ ]:
# One-hot-encoding.
y = np.concatenate([y_train,y_test],axis=0)
y = to_categorical(y,10)
y_train = y[:n_train_size,:]
y_test = y[n_train_size:,:]

#### 1.4. Download a pre-trained CNN model and prepare it:

In [ ]:
# Use only the convolutional base: include_top = False.
# input_shape = (32,32,3) to conform with the image size.
my_conv = MobileNet(include_top=False, weights='imagenet',input_shape = (32,32,3))

In [ ]:
# Print out the pre-trained layers.
i = 0
for my_layer in my_conv.layers:
    print("Layer #{} : '{}' is trainable = {}".format(i,my_layer.__class__.__name__, my_layer.trainable))
    i +=1

In [ ]:
# Turn the trainable property to False => Freeze the weights of the convolutional base.
# This would be necessary if you want to just "fine-tune".
# If you want to retrain the whole network, leave the following two lines as comments.

#for my_layer in my_conv.layers:
#    my_layer.trainable = False

In [ ]:
# Now, show a summary.
my_conv.summary()

#### 1.5. Now, attach the classification layers on top of the convolutional base:

In [ ]:
# Get the input and output of the convolutional base.
my_input = my_conv.input
my_output = Flatten()(my_conv.output)

In [ ]:
# Build the rest of the output layers.
drop_prob = 0.5
# Attach a fully connected layer.
my_output = Dense(units = 512, activation="relu")(my_output)             # 512 comes from running summary() function.
# Apply dropout.
my_output = Dropout(rate=drop_prob)(my_output)
# The output layer.
my_output = Dense(units = 10, activation="softmax")(my_output)           # There are 10 labels.
# Finally, use the Functional API to build the model.
my_model = Model(inputs=my_input,outputs=my_output)                      

In [ ]:
# View the summary of the entire fine-tuning model.
my_model.summary()

#### 1.6. Define the hyperparameters and optimizer:

In [ ]:
# Hyperparameters.
n_epochs = 10
batch_size = 10
learn_rate = 0.0001

In [ ]:
# Define the optimizer and then compile.
my_optimizer=Adam(lr=learn_rate)
my_model.compile(loss = "categorical_crossentropy", optimizer = my_optimizer, metrics=["accuracy"])

#### 1.7. Train the model and visualize the history:

In [ ]:
# Train the model.
# verbose = 0 means no output. verbose = 1 to view the epochs.
my_summary = my_model.fit(X_train, y_train, epochs=n_epochs, batch_size = batch_size, validation_split = 0.3, verbose = 1)

In [ ]:
# Visualize the training history. 
n_skip = 0                 # Skip the first few steps.
plt.plot(my_summary.history['accuracy'][n_skip:], c="b")
plt.plot(my_summary.history['val_accuracy'][n_skip:], c="g")
plt.title('Training History')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

#### 1.8. Testing: 

In [ ]:
ACC = my_model.evaluate(X_test, y_test, verbose=0)[1]    
print("Test Accuracy : {}".format(np.round(ACC,3)))